In [50]:
# Libraries used to train models & manipulate data
import numpy as np
import pandas as pd

### Kaggle import: https://github.com/Kaggle/kaggle-api
# kaggle datasets download -f dump.csv --unzip killbot/linkedin-profiles-and-jobs-data
members = pd.read_csv(r'./dump.csv')
# pos_vecs = pd.read_csv(r'./tfidf_positions.csv', header=None) # NN input data
# Filter for useful entries 
members = members.filter(items=['memberUrn', 'posTitle'])
members.head(3)

,memberUrn,posTitle
0,urn:li:member:10013136,Portfolio Executive
1,urn:li:member:10013136,Solution Delivery Executive
2,urn:li:member:10013136,Project Manager


In [51]:
# Simplify the member id
members['memberUrn'] = members['memberUrn'].apply(lambda x: int(x.split(':')[-1]))

print(members.shape)
members.head(3)

(39537, 2)


,memberUrn,posTitle
0,10013136,Portfolio Executive
1,10013136,Solution Delivery Executive
2,10013136,Project Manager


In [52]:
members = members[members.groupby('posTitle')['posTitle'].transform('count') > 1] # Only values that have multiple inputs
members = members.drop_duplicates()
members.shape

(15517, 2)

In [53]:
# Group members by position
members_grouped = members.groupby('posTitle')['memberUrn'].apply(list) # 2817 jobs w/ more than 1 member
# Series -> DataFrame & Reset indices
members_grouped = members_grouped.to_frame()
members_grouped = members_grouped.reset_index()

print(members_grouped.shape)
members_grouped.head(3)

(2817, 2)


,posTitle,memberUrn
0,.,"[224932880, 309331763]"
1,.NET Developer,"[100497522, 11628672, 16096244, 175387447, 222..."
2,.NET Software Developer,"[48796961, 65374720]"


In [54]:
#! HORRIBLE MEMORY
ids = members['memberUrn'].unique().tolist()

def multiEncodeMembers(urnList):
    l = [0]*len(ids)

    for urn in urnList:
        if urn:
            index = ids.index(urn)
            l[index] = 1
    return l

# Multi=encode the members
members_grouped['memberUrn'] = members_grouped['memberUrn'].apply(multiEncodeMembers)
# Get rid of entries that only appear once
# members_grouped = members_grouped[members_grouped.posTitle.duplicated(keep=False)] # No values?
# Drop duplicate entries now
# members_grouped['posTitle'] = members_grouped['posTitle'].drop_duplicates()

print(members_grouped.shape)
members_grouped.head(3)

(2817, 2)


,posTitle,memberUrn
0,.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,.NET Developer,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,.NET Software Developer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [56]:
#! HORRIBLE MEMORY
pos_names = members['posTitle'].unique().tolist()

# Multi=encode the members
pos_encoded = members_grouped['posTitle'].apply(lambda x: pos_names.index(x)).to_frame()
pos_encoded = pos_encoded.rename(columns={'posTitle': 'posEncoded'})
# Group data together
train = pd.concat([members_grouped, pos_encoded], axis=1)
train = train.dropna()

train.to_json('w2v_train.json')

print(len(ids))
print(len(pos_names))
print(train.shape)
train.head(5)

5954
2817
(2817, 3)


,posTitle,memberUrn,posEncoded
0,.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2018
1,.NET Developer,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",22
2,.NET Software Developer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2682
3,.NET Technical Lead,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2779
4,.Net Developer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",250
